In [ ]:
from pathlib import Path

import trio # type: ignore

from lean_client.trio_server import TrioLeanServer


async def main():
    lines = Path('lean-client-python/examples/test.lean').read_text().split('\n')

    async with trio.open_nursery() as nursery:
        server = TrioLeanServer(nursery, debug=True, debug_bytes=True)
        await server.start()
        await server.full_sync('lean-client-python/examples/test.lean')

        for i, line in enumerate(lines):
            before = await server.state('lean-client-python/examples/test.lean', i+1, 0)
            after = await server.state('lean-client-python/examples/test.lean', i+1, len(line))
            if before or after:
                print(f'Line {i+1}: {line}')
                print(f'State before:\n{before}\n')
                print(f'State after:\n{after}\n')

        server.kill()
        nursery.cancel_scope.cancel()

if __name__ == '__main__':
    trio.run(main)

Init TrioLeanServer
Sending SyncRequest(file_name='lean-client-python/examples/test.lean', content=None)
Sending b'{"file_name": "lean-client-python/examples/test.lean", "seq_num": 1, "command": "sync"}\n'
entering receiver function
receiver 1


Watchdog error: Cannot read LSP request: A Lean 3 request was received. Please ensure that your editor has a Lean 4 compatible extension installed. For VSCode, this is

    https://github.com/leanprover/vscode-lean4 


In [1]:
from pathlib import Path

lines = Path('lean-client-python/examples/test.lean').read_text().split('\n')

In [2]:
lines

['import Mathlib.Data.Set.Basic',
 'import Mathlib.Data.Real.Basic',
 'import Mathlib.Tactic.Basic',
 'import Mathlib.Tactic.Linarith',
 'import Mathlib.Data.Complex.Exponential',
 'import Mathlib.Analysis.SpecialFunctions.Log.Basic',
 '',
 'variable {α : Type*}',
 'variable (s t u : Set α)',
 'open Set',
 '',
 'example (h : s ⊆ t) : s ∩ u ⊆ t ∩ u := by',
 '  rw [subset_def, inter_def, inter_def]',
 '  rw [subset_def] at h',
 '  simp only [mem_setOf]',
 '  rintro x ⟨xs, xu⟩',
 '  exact ⟨h _ xs, xu⟩',
 '  ',
 'def main : IO Unit :=',
 '  IO.println s!"Hello!"']

In [3]:
from lean_client.commands import SyncRequest

request = SyncRequest('lean-client-python/examples/test.lean')

In [4]:
request

SyncRequest(file_name='lean-client-python/examples/test.lean', content=None)

In [5]:
from lean_client.commands import CommandResponse

CommandResponse.parse_response('import Mathlib.Data.Set.Basic')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
import json
import subprocess

def send_lsp_request(lsp_command):
    # Start the Lean server as a subprocess
    process = subprocess.Popen(['lean', '--server'],
                               stdin=subprocess.PIPE,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               text=True)

    print(json.dumps(lsp_command))
    # Send an LSP request to the Lean server
    process.stdin.write(json.dumps(lsp_command) + "\n")
    process.stdin.flush()

    # Read the response from Lean server
    response = process.stdout.readline()
    print("Response from Lean server:", response)

    # Close the Lean server
    process.terminate()
    process.wait()

# Example LSP request for initialization (simplified version)
# lsp_request = {
#     "jsonrpc": "2.0",
#     "id": 1,
#     "method": "initialize",
#     "params": {
#         "capabilities": {},  # Capabilities can be filled as needed
#         "processId": None,
#         "rootPath": None,
#         "rootUri": None
#     }
# }

lsp_request = {"jsonrpc":"2.0","method":"textDocument/didOpen","params":{"textDocument":{"uri":"file:///test.lean","languageId":"lean","version":1,"text":"import Init.System.IO\n\ndef n : Nat := 1234\n\n#check n\n\ndef s : String := \"abcd\"\n\n#check s\n\ndef hello : IO Unit := IO.println \"Hello world!\"\n\n#eval hello\n\ndef αβγ /- 😉😉😉😉 -/: Nat := \"NotANat\"\n\nnamespace MyNs\n\ndef u : Unit := ()\n\nend MyNs\n\n#check MyNs.u\n\n#print MyNs.u\n"}}}

send_lsp_request(lsp_request)


{"jsonrpc": "2.0", "method": "textDocument/didOpen", "params": {"textDocument": {"uri": "file:///test.lean", "languageId": "lean", "version": 1, "text": "import Init.System.IO\n\ndef n : Nat := 1234\n\n#check n\n\ndef s : String := \"abcd\"\n\n#check s\n\ndef hello : IO Unit := IO.println \"Hello world!\"\n\n#eval hello\n\ndef \u03b1\u03b2\u03b3 /- \ud83d\ude09\ud83d\ude09\ud83d\ude09\ud83d\ude09 -/: Nat := \"NotANat\"\n\nnamespace MyNs\n\ndef u : Unit := ()\n\nend MyNs\n\n#check MyNs.u\n\n#print MyNs.u\n"}}}
Response from Lean server: 
